Kannan: This notebook is for stitching various features generated from featuer engineering step (both numerical and textual features) and run feature significance analysis and select K-best features to feed to ML model.

In [96]:
#importing libraries
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet

from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso

In [97]:
#supress outliers
outlier_max_value = 25000
outlier_flag=True

### Reading data & Stitching features

In [98]:
custom_features=True

if custom_featrues:
#File we received from professors
inputFile = '../data/input/OnlineNewsPopularity.csv'
df = pd.read_csv(inputFile)

#file we re-engineered
inputFile = '../data/output/0_recreated_basic_features.xlsx'
df = pd.read_csv(inputFile)



df.columns = df.columns.str.replace(' ','')
df.head()

,Id,url,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,num_keywords,data_channel_is_lifestyle,data_channel_is_entertainment,data_channel_is_bus,data_channel_is_socmed,data_channel_is_tech,data_channel_is_world,shares
0,1,http://mashable.com/2014/09/01/americans-held-...,10,261,0.661355,1.0,0.787500,7,3,1,1,4.873563,7,0,0,0,0,0,1,1100
1,2,http://mashable.com/2014/09/01/apple-visa-mast...,7,1791,0.370242,1.0,0.535038,74,3,50,0,4.554439,8,0,0,0,0,0,0,1100
2,3,http://mashable.com/2014/09/01/aussie-football...,7,503,0.524291,1.0,0.704918,3,3,1,0,5.003976,5,0,0,0,0,0,1,1000
3,4,http://mashable.com/2014/09/01/australia-gover...,10,526,0.536204,1.0,0.654867,17,1,1,0,4.998099,9,0,0,0,0,0,1,822
4,5,http://mashable.com/2014/09/01/australia-jane-...,13,237,0.619048,1.0,0.807143,5,3,1,0,5.046414,9,0,0,0,0,0,1,841


In [99]:
#List of feature generated files
filepath='../data/output/'
files=[
    '2_NLP_Clustering.xlsx',
    '2_Sentiment_Polarity_Subjectivity.xlsx',
    '2_NLP_LDA_NMF_LSI_lite.xlsx',
    '2_d2v_keywords.xlsx',
    '2_d2v_content.xlsx',
    '2_d2v_NER_list.xlsx',
    '2_NER_Type_Count.xlsx',
    '2_Day_of_Week_Extraction.xlsx',
    '2_keywords_popularity_prediction.xlsx'
]

In [100]:
%%time
#Merging feature files
for f in files:
    df_tmp=pd.read_excel(filepath+f)
    if f.find('_d2v_')>0:
        df_tmp = df_tmp.iloc[:,0:11] 
    df=pd.merge(df,df_tmp,how='left',on='Id')
    print('file:',f,'\tColumns:',len(df_tmp.columns))

file: 2_NLP_Clustering.xlsx 	Columns: 11
file: 2_Sentiment_Polarity_Subjectivity.xlsx 	Columns: 5
file: 2_NLP_LDA_NMF_LSI_lite.xlsx 	Columns: 21
file: 2_keywords_popularity_prediction.xlsx 	Columns: 2
Wall time: 2.85 s


In [101]:
#Selecting columns (excluding vector columns; keeping only pca columns instead)
cols=df.columns.tolist()
cols.append(cols.pop(cols.index('shares')))#moving target column to the end
df=df[cols]

df_raw=df.copy()
df=df.drop('url',axis=1)
df=df.drop('Id',axis=1)
df.head()

,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,num_keywords,data_channel_is_lifestyle,data_channel_is_entertainment,data_channel_is_bus,data_channel_is_socmed,data_channel_is_tech,data_channel_is_world,cluster_0,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,cluster_7,cluster_8,cluster_9,polarity_title,subjectivity_title,polarity_content,subjectivity_content,LDA_C0,LDA_C1,LDA_C2,LDA_C3,LDA_C4,LDA_C5,LDA_C6,LDA_C7,LDA_C8,LDA_C9,LDA_K0,LDA_K1,LDA_K2,LDA_K3,LDA_K4,LDA_K5,LDA_K6,LDA_K7,LDA_K8,LDA_K9,keyword_predicted_shares,shares
0,10,261,0.661355,1.0,0.787500,7,3,1,1,4.873563,7,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0.000,0.00,0.013542,0.390972,0.000248,0.034725,0.022323,0.708774,0.147222,0.071431,0.000248,0.014535,0.000248,0.000248,0.025000,0.025000,0.025000,0.774996,0.025000,0.025000,0.025000,0.025000,0.025000,0.025004,1931,1100
1,7,1791,0.370242,1.0,0.535038,74,3,50,0,4.554439,8,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0.000,0.00,0.056136,0.518788,0.000763,0.000763,0.099631,0.000763,0.188261,0.019300,0.000764,0.688228,0.000763,0.000764,0.014287,0.014286,0.161571,0.014286,0.724142,0.014286,0.014286,0.014286,0.014286,0.014286,2167,1100
2,7,503,0.524291,1.0,0.704918,3,3,1,0,5.003976,5,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,-0.075,0.05,0.003831,0.348095,0.219737,0.100384,0.000568,0.043743,0.000568,0.195224,0.190030,0.019347,0.229831,0.000568,0.025000,0.025000,0.025000,0.524997,0.025000,0.025000,0.025000,0.274996,0.025000,0.025007,1978,1000
3,10,526,0.536204,1.0,0.654867,17,1,1,0,4.998099,9,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0.000,0.00,0.124162,0.378684,0.000490,0.000490,0.000490,0.000490,0.140903,0.439507,0.000490,0.184880,0.000490,0.231768,0.025000,0.025000,0.025000,0.524998,0.025000,0.025000,0.025000,0.274999,0.025000,0.025003,1924,822
4,13,237,0.619048,1.0,0.807143,5,3,1,0,5.046414,9,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0.000,0.75,0.070928,0.486269,0.008966,0.089297,0.301626,0.057744,0.089295,0.054065,0.397762,0.000415,0.000415,0.000415,0.033333,0.033333,0.033333,0.699996,0.033333,0.033333,0.033333,0.033333,0.033333,0.033337,2013,841


In [102]:
print(df.shape)

(7795, 53)


In [103]:
#supress_outiers
if outlier_flag:
    df['shares'] = df['shares'].clip(upper = outlier_max_value)

In [104]:
#selection input features and target column
X=df.drop('shares',1) #Feature matrix
y=df['shares'] #target column

In [105]:
print(X.shape)
print(y.shape)

(7795, 52)
(7795,)


## #2: Wrapper Method

A wrapper method uses a ML algorithm and its performance as evaluation criteria. we feed the features to the ML algorithm and based on the model performance we add/remove the features. This is an iterative and computationally expensive process but it is more accurate than the filter method.

Wrapper methods
- Backward Elimination
- Forward Selection
- Bidirectional Elimination
- Recursive Feature Elimination (RFE)

We implemented Backward Elimination and RFE here.

In [106]:
# Display all columns and rows in pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
#Recursive feature elimination
model = LinearRegression()

#no of features
nof_list=np.arange(1,len(X.columns))         
high_score=0
#Variable to store the optimum features
nof=0           
score_list =[]
for n in range(len(nof_list)):
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 0)
    model = LinearRegression()
    rfe = RFE(model,nof_list[n])
    X_train_rfe = rfe.fit_transform(X_train,y_train)
    X_test_rfe = rfe.transform(X_test)
    model.fit(X_train_rfe,y_train)
    score = model.score(X_test_rfe,y_test)
    score_list.append(score)
    if(score>high_score):
        high_score = score
        nof = nof_list[n]
print("Optimum number of features: %d" %nof)
print("Score with %d features: %f" % (nof, high_score))

In [110]:
model = LinearRegression()
cols = list(X.columns)
#Initializing RFE model
rfe = RFE(model, nof)             
#Transforming data using RFE
X_rfe = rfe.fit_transform(X,y)  
#Fitting the data to model
model.fit(X_rfe,y)              
temp = pd.Series(rfe.support_,index = cols)
selected_features_rfe = temp[temp==True].index
print(selected_features_rfe)

Index(['n_tokens_title', 'n_unique_tokens', 'n_non_stop_words',
       'n_non_stop_unique_tokens', 'num_hrefs', 'num_self_hrefs', 'num_imgs',
       'num_videos', 'average_token_length', 'num_keywords',
       'data_channel_is_lifestyle', 'data_channel_is_entertainment',
       'data_channel_is_bus', 'data_channel_is_socmed', 'data_channel_is_tech',
       'data_channel_is_world', 'cluster_0', 'cluster_1', 'cluster_2',
       'cluster_3', 'cluster_4', 'cluster_5', 'cluster_6', 'cluster_7',
       'cluster_8', 'cluster_9', 'polarity_title', 'subjectivity_title',
       'polarity_content', 'subjectivity_content', 'LDA_C0', 'LDA_C1',
       'LDA_C2', 'LDA_C3', 'LDA_C4', 'LDA_C5', 'LDA_C6', 'LDA_C7', 'LDA_C8',
       'LDA_C9', 'LDA_K0', 'LDA_K1', 'LDA_K2', 'LDA_K3', 'LDA_K4', 'LDA_K5',
       'LDA_K6', 'LDA_K7', 'LDA_K8', 'LDA_K9', 'keyword_predicted_shares'],
      dtype='object')


## Writing output files with Stitiched Features

In [111]:
def write_selected_features(overall_selected_features,method):
    overall_selected_features.append('Id')
    overall_selected_features.append('url')
    overall_selected_features.append('shares')
    df_selected=df_raw[overall_selected_features]
    df_selected.to_csv(filepath+'OnlineNewsPopularity_featureEngg_'+method+'.csv',index=False)
    return

### Recursive feature elimitation

In [113]:
#Selected Features : Recursive feature Elimination
write_selected_features(list(selected_features_rfe),'RecEli')
#list(selected_features_rfe)

# Open the file for writing
F = open('../data/output/Feature_ranking_RFE.txt', 'w')
F.write("\n".join([str(x) for x in list(selected_features_rfe)]))
F.close()

### RAW

In [114]:
df_raw.to_csv(filepath+'OnlineNewsPopularity_featureEngg_RAW.csv',index=False)